

# Importing libraries and defining errors



In [ ]:
from IPython.display import clear_output 
import numpy as np

In [ ]:
class Error(Exception):
  '''Base class for user defined errors'''
  pass

In [ ]:
class keyInverseError(Error):
  '''Raised when inverse of key does not exist'''
  pass

In [ ]:
class NoTextEntered(Error):
  '''Raised when user enter a empty text'''
  pass

In [ ]:
class KeyOutOfRange(Error):
  '''Raised an error when key is out of domain'''

# GCD and Multiplicative Inverse function

In [ ]:
def gcd(a:int,b:int):
  '''greatest common divisor function will return the gcd of two numbers a and b using basic eucleadean theorem.'''
  if a==0:
    return b
  else:
    return gcd(b%a,a)

In [ ]:
def mult_inverse(A:int,B:int):
  '''
    multiplicative inverse function will take two arguments A and B and will find the multiplicative inverse of them using extended eucleadean theorem.
  '''
  if(gcd(abs(A),B)!=1):
    return -1
  else:
    a,b=max(A,B),min(A,B)
    t1=0
    t2=1
    q=int(a/b)
    r=a%b
    t=t1-(t2*q)
    #print('q a b r t1 t2 t\n')
    #print(q,a,b,r,t1,t2,t,"\n")
    while(r!=0):
      a=b
      b=r
      t1=t2
      t2=t
      q=int(a/b)
      r=a%b
      t=t1-(t2*q)
      #print(q,a,b,r,t1,t2,t,"\n")
    if(t2<0):
      return t2+max(A,B)
    else:
      return t2

In [ ]:
s_alpha = dict()
A=range(0,26)
B=range(97,123)
s_alpha={x:chr(y) for (x,y) in zip(A,B)}

In [ ]:
def get_alpha(val:int,my_dict:dict()):
  '''get_key function take value and dictionary as argument and return the key of the value in that dictionary'''
  for key, value in my_dict.items():
    if val == value:
      return key
  return "key doesn't exist"

#Affine cipher

In [ ]:
def affine_cipher(key_1:int,key_2:int,text:str,type:int=1):
  '''
     affine_cipher function 
     converted text = (key_1*text + type*key_2)%26
     it takes 
     key_1->Key for multiplication,
     key_2->Key for addition. 
     text->Cipher text in case of decryption and Plain text in case of encryption 
     type-> 1 for encryption , -1 for decryption.
  '''
  text = text.lower()
  converted_text = ""
  if type ==-1:
    key_2 = key_2*key_1
  for x in text:
    if(x in ['0','1','2','3','4','5','6','7','8','9']):
      converted_text+=x
    elif(x.islower()):
      p = get_alpha(x,s_alpha)
      index = ((key_1*p)+(type*key_2))%26
      converted_text+=s_alpha[index]
  return converted_text

In [ ]:
def key_inp(message:str,type:int=1):
  '''key_inp will take type as parameter
      if type=0, then it will return a valid key_2 
      if type=1, then it will return a valid key_1 for encryption.
      if type=2, then it will return inverse of a valid key_1 for decryption
  '''
  try:
    key=int(input(message))
    if key not in range(0,26):
      #if key is not between 0 to 25
      raise KeyOutOfRange
    if type!=0:
      keyI = mult_inverse(key,26)
      if keyI==-1:
        #if inverse of key does not exist
        raise keyInverseError
      else:
        if type==1:
          return key
        else:
          return keyI
    else:
      return key
  except keyInverseError:
    print('Invalid key because inverse of key does not exist')
    return key_inp(message,type)
  except KeyOutOfRange:
    print('Key should be in range of 0 to 25.')
    return key_inp(message,type)
  except ValueError:
    #if user enter anything except integer
    print('Invalid input')
    return key_inp(message,type)

In [ ]:
def message_inp(message:str):
  '''message_inp function will take message for input taking screen as parameter and returns the valid message'''
  try:
    text = input(message)
    if(text==""): #if user entered a empty message
      raise NoTextEntered
    else:
      return text.lower()
  except NoTextEntered:
    print("You did not entered anything. Please try Again !!!")
    return message_inp(message)

#Cryptanalysis of affine cipher

In [ ]:
def crypt_analysis_affine_cipher(text: str,pt:str='',ct:str=''):
  '''crypt_analysis_mult_cipher function will take ciphered text as parameter and return all possible plain text with valid keys'''
  for i in range(0,26):
    keyI = mult_inverse(i,26) #finding multiplicative inverse of key
    if(keyI==-1):
      #if inverse does not exist for a key
      continue
    else:
      #if inverse exist for a key
      if (pt=='' or ct==''):
        #if no pt-ct pair are given
        for j in range(0,26):
          print(f"for key_1 = {i} & key_2 = {j}, plain text is : {affine_cipher(keyI,j,text,-1)}")
      elif(len(pt)==1):
        #if only single letter ct-pt pair given
        for j in range(0,26):
          if(affine_cipher(keyI,j,ct,-1)==pt):
            print(f"for key_1 = {i} & key_2 = {j}, plain text is : {affine_cipher(keyI,j,text,-1)}")
      else:
        decoding_CT_PT_pair(text,pt,ct)
        break

In [ ]:
def decoding_CT_PT_pair(text:str,pt:str,ct:str):
  '''Function to decode the given ct-pt pair.
      it will take:
      text->cipher text
      pt,ct->given pt-ct pair
      it will calculate key_1 and key_2 with the help of ct-pt pair and return the deciphered text or plain text.
  '''
  pt,ct=pt.lower(),ct.lower()
  sol = eq_solver(get_alpha(pt[0],s_alpha),get_alpha(pt[1],s_alpha),get_alpha(ct[0],s_alpha),get_alpha(ct[1],s_alpha))
  print(f"Plain text after cryptoanalysis: {affine_cipher(mult_inverse(sol[0],26),sol[1],text,-1)}")

In [ ]:
def eq_solver(x1:int,x2:int,y1:int,y2:int):
  '''eq_solver function to solve equation:
        y1 = (key_1 * x1) + key_2
        y2 = (key_1 * x2) + key_2 
        it will return the solution as numpy array [key_1,key_2]
  '''
  A = np.array([[x1,1],[x2,1]])
  b = np.array([y1,y2])
  return np.linalg.solve(A, b)

#Main program

In [ ]:
def menu():
  '''menu function to print menu of our program.'''
  print("~~~~~~~~~~~~Menu~~~~~~~~~~~~")
  print("1. Encrypt your message using affine cipher.")
  print("2. Decrypt your message using affine cipher.")
  print("3. Cryptoanalysis of affine cipher.")
  print("4. Exit")

In [ ]:
def main():
  #driver function
  while(True):
    menu() #printing menu
    ch=input("Enter your choice: ")
    
    #checking if user entered a valid choice
    while ch not in ['1','2','3','4']:
      ch=input("Wrong choice, Enter again: ")

    #clearing screen for upcoming operation
    clear_output()

    #if user wants to exit
    if ch=='4':      
      print("GoodBye!!!")
      break
    else:

      #encryption using affine cipher
      if ch=='1':
        #taking key_1 from user for encryption
        key_1 = key_inp("Enter key_1 : ",1)
        #taking key_2 from user for encryption
        key_2 = key_inp("Enter key_2 : ",0) 
        #taking message from user for encryption
        text = message_inp("Enter your plain text: ")
        print("Your ciphered text is :",affine_cipher(key_1,key_2,text))
      
      #decryption using affine cipher
      elif ch=='2':
        #taking key from user for decryption
        key_1 = key_inp("Enter key_1 : ",2)
        key_2 = key_inp("Enter key_2 : ",0)
        #taking message from user for decryption
        text = message_inp("Enter your ciphered text: ")
        print("Your plain text is :",affine_cipher(key_1,key_2,text,-1))
      
      #cryptoanalysis of ciphered text using affine cipher
      else:
        #taking cipher text from user for cryptoanalysis
        text = message_inp("Enter your cipher text for cryptoanalysis: ")
        user_ch = message_inp("Do you have any ct-pt pair ? (Y/N): ").lower()
        if user_ch =='y' or user_ch == '0' or user_ch =='yes':
          pt = message_inp("Enter plain text of known pt-ct pair: ").lower()
          ct = message_inp("Enter cipher text of known pt-ct pair: ").lower()
          if(len(pt)==len(ct)):
            crypt_analysis_affine_cipher(text,pt,ct)
          else:
            n=min(len(pt),len(ct))
            crypt_analysis_affine_cipher(text,pt[0:n],ct[0:n])
        else:
          crypt_analysis_affine_cipher(text)

In [ ]:
main() #calling of driver function

GoodBye!!!
